In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import io
import csv
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stats import mean, median, mode, multi_mode

ModuleNotFoundError: No module named 'stats'

In [ ]:
url = "https://opendata.arcgis.com/datasets/8070cd333824453abea19e8faa340fb8_0.geojson"
response = requests.get(url).json()

with open('lightrail_data.txt', 'w') as outfile:
   json.dump(response, outfile)
with open('lightrail_data.txt') as rail_file:
   dict_rail = json.load(rail_file)

rail_data = pd.DataFrame.from_dict(json_normalize(dict_rail['features']), orient='columns')
rail_data.head()

In [ ]:
# Reorganizing the columns using double brackets
organized_rail_df = rail_data[['properties.StationName', 
                               'properties.Q2015_1', 
                               'properties.Q2015_2', 
                               'properties.Q2015_3', 
                               'properties.Q2015_4', 
                               'properties.Q2016_1',
                               'properties.Q2016_2',
                               'properties.Q2106_3',
                               'properties.Q2016_4',
                               'properties.Q2017_1',
                               'properties.Q2017_2',
                               'properties.Q2017_3',
                               'properties.Q2017_4',
                               'properties.Q2018_1',
                               'geometry.coordinates', 
                               'properties.Jurisdiction',
                               'properties.Type']]
organized_rail_df.head()

In [ ]:
renamed_rail_df = organized_rail_df.rename(columns={'properties.StationName': 'Station Name', 
                                               'properties.Q2015_1': '2015 Q1', 
                                               'properties.Q2015_2': '2015 Q2', 
                                               'properties.Q2015_3': '2015 Q3', 
                                               'properties.Q2015_4': '2015 Q4', 
                                               'properties.Q2016_1': '2016 Q1',
                                               'properties.Q2016_2': '2016 Q2',
                                               'properties.Q2106_3': '2016 Q3',
                                               'properties.Q2016_4': '2016 Q4',
                                               'properties.Q2017_1': '2017 Q1',
                                               'properties.Q2017_2': '2017 Q2',
                                               'properties.Q2017_3': '2017 Q3',
                                               'properties.Q2017_4': '2017 Q4',
                                               'properties.Q2018_1': '2018 Q4',
                                               'geometry.coordinates': 'Stop Locations', 
                                               'properties.Jurisdiction': 'Local Municipality',
                                               'properties.Type': 'Eastbound or Westbound'})
renamed_rail_df.head()

In [ ]:
# Replacing Boadrings and Departures with proper directional naming conventions
renamed_rail_df.loc[:,'Eastbound or Westbound'] = renamed_rail_df['Eastbound or Westbound'].str.replace('Boardings', 'Eastbound').str.replace('Departures', 'Westbound')  
renamed_rail_df

In [ ]:
# Removing brackets from geometry.coordinates, splitting into lat/lon columns
renamed_rail_df['Locations'] = renamed_rail_df['Stop Locations'].astype(str).str.replace('\[|\]','')

# # Creating lat/lon lists
lat = []
lon = []
for row in renamed_rail_df['Locations']:
   lat.append(row.split(',')[0])
   lon.append(row.split(',')[1])
    
# appending lat and lon data to new columns in dataframe
renamed_rail_df['Latitude'] = lat
renamed_rail_df['Longitude'] = lon

# Dropping unnecessary columns
renamed_rail_df = renamed_rail_df.drop(columns=['Stop Locations', 'Locations'])
renamed_rail_df.head()

In [ ]:
# Which quarters do we see an increase in light rail riders from 2015 to 2017? 


In [ ]:
# Getting data from API, turning into json file
url = "https://municipal.systems/v1/places/az/dataTypes/shared-vehicle/data?key=f867b00f-7408-4c88-bd4d-ad4ef582b20c"
response2 = requests.get(url).json()
# Creating a .txt file to be read into a dataframe
with open('bike_data.txt', 'w') as outfile2:
  json.dump(response2, outfile2)
with open('bike_data.txt') as bike_file:
  dict_bike = json.load(bike_file)
# Turning json file into a readable dataframe
bike_data = pd.DataFrame.from_dict(json_normalize(dict_bike['results']), orient='columns')

# Creating editable dataframe, formatting column names
bike_data_df = bike_data
bike_data_df.columns = bike_data_df.columns.str.strip().str.lower().str.replace('data.','')

# Dropping unnecessary columns
bike_data_df = bike_data_df[['name','geometry.coordinates','operators']]

# Removing brackets from operators values, then dropping JUMP LA rows
bike_data_df.loc[:,'operators'] = bike_data_df['operators'].str.get(0)
bike_data_df = bike_data_df[bike_data_df.operators != 'JUMP LA']
# Removing brackets from geometry.coordinates, splitting into lat/lon columns
bike_data_df['coordinates'] = bike_data_df['geometry.coordinates'].astype(str).str.replace('\[|\]','')

# # Creating lat/lon lists
lat = []
lon = []
for row in bike_data_df['coordinates']:
   lat.append(row.split(',')[0])
   lon.append(row.split(',')[1])
    
# appending lat and lon data to new columns in dataframe
bike_data_df['latitude'] = lat
bike_data_df['longitude'] = lon

# Dropping unnecessary columns
bike_data_df = bike_data_df[['name','operators','latitude','longitude']]
bike_data_df.head()

In [ ]:
csv_bus_stops = "https://raw.githubusercontent.com/Sephka/Project1/master/Project_Folder/phx_city_data/bus_stops"
bus_stops_df = pd.read_csv(csv_bus_stops, encoding="utf-8")
reduced_df = bus_stops_df[["stop_id", "stop_name", "stop_lat", "stop_lon","district"]]
reduced_df.head()

In [ ]:
# Street closure data
csv_street_closures = "https://raw.githubusercontent.com/Sephka/Project1/Kristen/Project1Data/traffic-restrictions_planned-street-restrictions-and-closures_trstreets_20190928.csv"
street_closures_df = pd.read_csv(csv_street_closures, encoding="utf-8")
street_closures_df.head()
street_closures_df = street_closures_df[["LOCATION", "DATES", "CLOSURE TYPE", "PURPOSE",]]
street_closures_df = reduced_df.applymap(lambda s:s.lower() if type(s) == str else s)
street_closures_df.columns = map(str.lower, reduced_df.columns)
street_closures_df.columns = reduced_df.columns.str.replace(" ","_")
street_closures_df.head()

In [ ]:
# Intersection closure data
csv_intersection_closures = "https://raw.githubusercontent.com/Sephka/Project1/Kristen/Project1Data/traffic-restrictions_planned-intersection-restrictions-and-closures_trint_20190928.csv"
intersection_closures_df = pd.read_csv(csv_intersection_closures, encoding="utf-8")
intersection_closures_df.head()
intersection_closures_df = intersection_closures_df[["LOCATION", "DATES", "CLOSURE TYPE", "PURPOSE",]]
intersection_closures_df = intersection_closures_df.applymap(lambda s:s.lower() if type(s) == str else s)
intersection_closures_df.columns = map(str.lower,intersection_closures_df.columns)
intersection_closures_df.columns = intersection_closures_df.columns.str.replace(" ","_")
intersection_closures_df.head()